In [4]:

import sys
import os
import sys
file = "/Users/charlesmiller/Documents/Code/dynamics_lab/DSDL/DLAESINDy/testcases/parallel_sweep.ipynb"
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(file))))

# sys.path.append("../aesindy")
from default_params import default_params
import numpy as np
from aesindy.solvers import DatasetConstructorMultiEquity
from aesindy.training import TrainModel
from aesindy.helper_functions import call_polygon
import wandb
from wandb.integration.keras import WandbMetricsLogger, WandbModelCheckpoint
import tensorflow as tf
import gc
import pandas as pd
from pysindy.differentiation import SmoothedFiniteDifference    


Exception in thread Exception in threading.excepthook:
Exception ignored in thread started by: <bound method Thread._bootstrap of <Thread(Thread-12, stopped 12931575808)>>
Traceback (most recent call last):
  File "/Users/charlesmiller/.pyenv/versions/3.9.18/lib/python3.9/threading.py", line 937, in _bootstrap
    self._bootstrap_inner()
  File "/Users/charlesmiller/.pyenv/versions/3.9.18/lib/python3.9/threading.py", line 982, in _bootstrap_inner
    self._invoke_excepthook(self)
  File "/Users/charlesmiller/.pyenv/versions/3.9.18/lib/python3.9/threading.py", line 1264, in invoke_excepthook
    local_print("Exception in threading.excepthook:",
  File "/Users/charlesmiller/.local/share/virtualenvs/dynamics_lab-5gulJLF-/lib/python3.9/site-packages/ipykernel/iostream.py", line 604, in flush
    self.pub_thread.schedule(self._flush)
  File "/Users/charlesmiller/.local/share/virtualenvs/dynamics_lab-5gulJLF-/lib/python3.9/site-packages/ipykernel/iostream.py", line 267, in schedule
    self.

In [5]:



def update_params_from_wandb(params, wandb_config):
    """
    Update the default parameters with values from the wandb config.
    
    Args:
    default_params (dict): The default parameter dictionary
    wandb_config (wandb.config): The wandb config object
    
    Returns:
    dict: Updated parameter dictionary
    """
    updated_params = params.copy()
    
    for key, value in wandb_config.items():
        if key in updated_params:
            updated_params[key] = value
        else:
            print(f"Warning: '{key}' found in wandb config but not in default params. Ignoring.")
    
    return updated_params



def model_runner(wandb_params, input_data):
    params = update_params_from_wandb(default_params, wandb_params)
    params['model'] = 'multi'
    params['case'] = 'hyp'
    params['use_wandb'] = True
    params['prediciton_mode'] = 'max'
    params['future_steps'] = 24
    params['open_close'] = 'open'
    
    # scaled_datasets = []
    # for data in input_data:
    data = input_data[int(params['data_length']*len(input_data)):]
        # scaled_data = scale_dataset(data, params)
        # scaled_datasets.append(scaled_data)
    ## slice the data based on a fractional proportion, must remain in sequential order
    # print(scaled_datasets[0])
    data_dict = {
        'x':data,
        'dt': 900
        }
    data_builder = DatasetConstructorMultiEquity(
                    params=params
                    )
    data_builder.build_solution(data_dict)
    train_data = data_builder.get_data()
    trainer = TrainModel(train_data, params)
    trainer.fit() 

    del trainer.model
    del trainer

    tf.keras.backend.clear_session()
    gc.collect()

def wandb_sweep(data):

    # Initialize the sweep
    sweep_id = 'cfm_inv/DLAESINDy/8gh00h9m'

    # Define the objective function for the sweep
    def sweep_train():

        wandb.init(project="DLAESINDy", config=wandb.config)
        model_runner(wandb.config, data)

    # Start the sweep
    wandb.agent(sweep_id, function=sweep_train, count=1000) 


In [6]:
datasets = []
# idxbig = ["QQQ","SPY","NVDA","IWM","AAPL","TSLA","MSFT","AMD","AMZN","META","GOOGL"]
# for symbol in idxbig:
# datasets.append(raw_data)
# scaled_data = raw_data.copy()
# scaled_data = scale_dataset(scaled_data, params={'input_dim': 256,'future_steps': 10})
raw_data = call_polygon("SPY",'2010-01-01','2024-09-01','minute',15)
raw_data['date'] = pd.to_datetime(raw_data['date']).dt.tz_localize(None)
raw_data['time'] = raw_data['date'].dt.time
raw_data.rename(columns={'c':'x'}, inplace=True)
wandb_sweep([raw_data])

wandb: Agent Starting Run: ejjgk036 with config:
wandb: 	batch_size: 32
wandb: 	coefficient_threshold: 2
wandb: 	data_length: 0.75
wandb: 	input_dim: 256
wandb: 	latent_dim: 2
wandb: 	learning_rate: 0.0001
wandb: 	loss_weight_integral: 0.05
wandb: 	loss_weight_layer_l1: 0.1
wandb: 	loss_weight_layer_l2: 0.1
wandb: 	loss_weight_prediction: 0.5
wandb: 	loss_weight_rec: 0.6
wandb: 	loss_weight_sindy_regularization: 1e-05
wandb: 	loss_weight_sindy_x: 0.0001
wandb: 	loss_weight_sindy_z: 0.001
wandb: 	loss_weight_x0: 0.05
wandb: 	n_frequencies: 2
wandb: 	poly_order: 2
wandb: 	sindy_init_scale: 5
wandb: 	sindy_threshold: 0.2
wandb: 	sindycall_freq: 500
wandb: 	smoothed_diff: False
wandb: 	smoothing_steps: 8
wandb: 	threshold_frequency: 100
wandb: 	widths_ratios: [0.8, 0.6, 0.4, 0.2]


wandb: 🚀 View run giddy-sweep-13 at: https://wandb.ai/cfm_inv/DLAESINDy/runs/3swa22qe
wandb: Find logs at: wandb/run-20241101_112909-3swa22qe/logs


wandb: WARNING Ignored wandb.init() arg project when running a sweep.


self.x.shape: (280, 17185)
self.dx.shape: (280, 17185)
results_202411011212_multi_hyp
generating data for sindycall
Fitting model..
--- Initial Sindy Coefficients ---
[[-0.01172875 -0.00204852]
 [ 0.00400059  0.02299615]
 [ 0.07150419 -0.0046026 ]
 [ 0.09566861 -0.0098974 ]
 [ 0.02196599 -0.00516358]
 [ 0.03874865 -0.0388257 ]
 [ 0.00138726 -0.00677848]
 [-0.03111202 -0.03631154]
 [-0.00613896 -0.10226157]
 [-0.09892759  0.01966307]
 [-0.03996806  0.11310082]
 [ 0.06193176  0.03122838]
 [ 0.01965855 -0.12105477]
 [-0.01898938 -0.07173982]]
Epoch 1/1500
430/430 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step - integral_loss: 2627145.2500 - l1_loss: 0.0389 - prediction_loss: 950.5971 - rec_loss: 8032.0186 - sindy_x_loss: 38046.7930 - sindy_z_loss: 285295.0000 - total_loss: 6837.3931 - x0_loss: 14813.2793
--- Running RFE ---
430/430 ━━━━━━━━━━━━━━━━━━━━ 149s 139ms/step - integral_loss: 2625013.0000 - l1_loss: 0.0389 - prediction_loss: 949.0687 - rec_loss: 8022.1885 - sindy_x_loss: 38073.6602 - sindy_z

wandb: Ctrl + C detected. Stopping sweep.


232/430 ━━━━━━━━━━━━━━━━━━━━ 18s 95ms/step - integral_loss: 545.9099 - l1_loss: 0.1269 - prediction_loss: 1.7500 - rec_loss: 1.6166 - sindy_x_loss: 39.6698 - sindy_z_loss: 24.3694 - total_loss: 2.1096 - x0_loss: 2.5927